In [4]:
from bs4 import BeautifulSoup
import urllib

url = 'https://en.wikipedia.org/wiki/Michigan_State_University'
import ssl

import pandas as pd



In [5]:
with urllib.request.urlopen(url, context=ssl.SSLContext(), timeout=5) as response:
    soup = BeautifulSoup(response.read())
    

In [6]:

all_a_tags_with_href = set(filter(lambda x: 'href' in x.attrs, soup.body.find_all('a')))

links_in_page = set(map(lambda x: x.attrs['href'], all_a_tags_with_href))

links_in_page = set(map(lambda x: urllib.parse.urljoin(url, x), links_in_page))
len(links_in_page)

links_in_wikipedia = set(filter(lambda x: urllib.parse.urlparse(x).netloc.endswith('wikipedia.org'), links_in_page))
len(links_in_wikipedia)



1800

In [7]:
webpages_on_wikipedia = set()

for wiki_url in links_in_wikipedia:
    try:
        with urllib.request.urlopen(wiki_url, context=ssl.SSLContext(), timeout=5) as response:
            content_type = response.info()['Content-Type'] 
            if 'text/html' in content_type:
                webpages_on_wikipedia.add(wiki_url)
    except:
        print(f'Unable to open {wiki_url}')

Unable to open https://en.wikipedia.org/w/index.php?title=Michigan_State_University&action=edit&section=40
Unable to open https://en.wikipedia.org/wiki/Michigan_State_University#cite_ref-sesquicentennial_16-2
Unable to open https://en.wikipedia.org/wiki/Michigan_State_University#cite_note-12
Unable to open https://en.wikipedia.org/wiki/G._Malcolm_Trout
Unable to open https://en.wikipedia.org/wiki/Michigan_State_University_Libraries
Unable to open https://en.wikipedia.org/wiki/Category:Use_mdy_dates_from_November_2020
Unable to open https://en.wikipedia.org/wiki/Dan_Gilbert_(businessman)
Unable to open https://en.wikipedia.org/wiki/Special:RecentChangesLinked/Michigan_State_University
Unable to open https://en.wikipedia.org/wiki/Michigan_State_University#19th_century
Unable to open https://en.wikipedia.org/wiki/American_conservative_movement
Unable to open https://en.wikipedia.org/wiki/Template_talk:Michigan_State_University
Unable to open https://en.wikipedia.org/wiki/Wikipedia:Content

In [9]:
col_names = ['source_url', 'link_url', 'link_title']
df = pd.DataFrame(columns = col_names)

links_in_wikipedia = webpages_on_wikipedia

i = -1

df_dict = {}

while len(df)<100 and i<100:

    i = i+1

    j = 0
    
    url = list(links_in_wikipedia)[i]

    with urllib.request.urlopen(url, context=ssl.SSLContext(), timeout=5) as response:
        soup = BeautifulSoup(response.read())

    all_a_tags_with_href = set(filter(lambda x: 'href' in x.attrs, soup.body.find_all('a')))

    links_in_page = set(map(lambda x: x.attrs['href'], all_a_tags_with_href))

    links_in_page = set(map(lambda x: urllib.parse.urljoin(url, x), links_in_page))

    links_in_link = set(filter(lambda x: urllib.parse.urlparse(x).netloc.endswith('wikipedia.org'), links_in_page))


    if len(links_in_link) >9:
        
        while j<10:
    
            wiki_url = list(links_in_link)[j]

            try:
                with urllib.request.urlopen(wiki_url, context=ssl.SSLContext(), timeout=5) as response:
                    content_type = response.info()['Content-Type'] 
                    if 'text/html' in content_type:
                        soup = BeautifulSoup(response.read())
            except:
                print(f'Unable to open {wiki_url}')

            df_dict = {'source_url': list(links_in_wikipedia)[i], 'link_url': list(links_in_link)[j] , 'link_title':soup.title.text}
            df = df.append(df_dict, ignore_index=True)
            j = j+1









/var/folders/tg/4c1kl49x0tjfvtgb52vdtt8h0000gn/T/ipykernel_51486/1337543341.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_dict, ignore_index=True)
/var/folders/tg/4c1kl49x0tjfvtgb52vdtt8h0000gn/T/ipykernel_51486/1337543341.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_dict, ignore_index=True)
/var/folders/tg/4c1kl49x0tjfvtgb52vdtt8h0000gn/T/ipykernel_51486/1337543341.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_dict, ignore_index=True)
/var/folders/tg/4c1kl49x0tjfvtgb52vdtt8h0000gn/T/ipykernel_51486/1337543341.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

In [13]:
df.to_csv('webcrawler.csv', sep = ',', index = False)